# <center>Бот, который сохраняет места для будущего посещения

In [1]:
import telebot
token = "780799099:AAGGjJfeKRiXX7D34_ZrW19n_zxOFcZbs70"

In [2]:
START, NAME, ADDRESS, PHOTO, LOCATION = range(5)

In [3]:
from collections import defaultdict
USER_STATE = defaultdict(lambda: START)

def get_state(message):
    return USER_STATE[message.chat.id]

def update_state(message, state):
    USER_STATE[message.chat.id] = state

In [23]:
PLACES = defaultdict(lambda: defaultdict(lambda: {}))
place_id = [0]

def get_places(user_id):
    return PLACES[user_id]  # TODO

def update_place(user_id, place_id, key, value):
    PLACES[user_id][place_id][key] = value
    
def update_photo_place(user_id, place_id, key, value):
    PLACES[user_id][place_id][key] = value  # TODO store the photo
    
def update_location_place(user_id, place_id, key, value):
    PLACES[user_id][place_id][key] = value  # TODO store the location
    
def update_place_id(place_id):
    place_id.append(place_id[-1] + 1)

In [25]:
bot = telebot.TeleBot(token)

@bot.message_handler(commands=["add"])
def handle_add(message):
    update_place_id(place_id)
    
    bot.send_message(message.chat.id, text="Напиши название")
    update_state(message, NAME)

@bot.message_handler(func=lambda message: get_state(message) == NAME)
def handle_name(message):    
    # название
    update_place(message.chat.id, place_id[-1], "name", message.text)
    bot.send_message(message.chat.id, text="Укажи адрес")
    update_state(message, ADDRESS)

@bot.message_handler(func=lambda message: get_state(message) == ADDRESS)
def handle_name(message):    
    # адрес
    update_place(message.chat.id, place_id[-1], "address", message.text)
    bot.send_message(message.chat.id, text="Загрузи фото")
    update_state(message, PHOTO)
    
@bot.message_handler(func=lambda message: get_state(message) == PHOTO)
def handle_name(message):   
    # фото
    update_photo_place(message.chat.id, place_id[-1], "photo", message.text)
    bot.send_message(message.chat.id, text="Загрузи локацию")
    update_state(message, LOCATION)
    
@bot.message_handler(func=lambda message: get_state(message) == LOCATION)
def handle_name(message):
    # локация
    update_location_place(message.chat.id, place_id[-1], "location", message.text)
    bot.send_message(message.chat.id, text="Место сохранено :)")
    update_state(message, START)

@bot.message_handler(commands=["list"])
def handle_list(message):
    # последние 10 добавленных мест
    places = PLACES[message.chat.id]
    places_all = sorted(list(places.items()), key=lambda x: x[0], reverse=True)
    places_last10 = places_all[:10]
    if places_last10 != []:
        text = f"{places_last10}"
    else:
        text = """
Список ваших мест пуст. 
Вы их можете начать добавлять с помощью команды /add
"""
    bot.send_message(message.chat.id, text=text)

@bot.message_handler(commands=["reset"])
def handle_reset(message):
    PLACES[message.chat.id] = defaultdict(lambda: {})
    
@bot.message_handler(commands=["start"])
def handle_list(message):
    text = """
Привет! Я Бот, который поможет тебе сохранить места для будущего посещения. 
Воспользуйся командой /help, чтобы узнать все доступные команды.
"""
    bot.send_message(message.chat.id, text=text)
    
@bot.message_handler(commands=["help"])
def handle_list(message):
    text = """
/add – добавление нового места
/list – отображение добавленных мест
/reset – удаление всех ваших добавленных локаций
"""
    bot.send_message(message.chat.id, text=text)

bot.polling()

In [10]:
place_id

[0, 1, 2]